## Web Scraping com a biblioteca BeautifulSoup

### O website utilizado será o site da [Imobiliária Imovirtual](https://www.imovirtual.com/comprar/apartamento/viseu/) e o navegador utilizado será o [Google Chrome](https://www.google.com/intl/pt-BR/chrome/).

***

In [1]:
# Importação de bibliotecas:

from bs4 import BeautifulSoup
import urllib.request as urllib_request
import pandas as pd
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import re

***

### 1. Requisição das informações:

In [2]:
# Realizando a requisição do HTML:

url = 'https://www.imovirtual.com/comprar/apartamento/viseu/'

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'}

try:
    
    req = Request(url, headers = headers, method = 'GET')
    response = urlopen(req)
    
except HTTPError as e:
    print(e.status, e.reason)
    
except URLError as e:
    print(e.reason)

In [3]:
# Criação de variáveis e do objeto BeautifulSoup:

html = response.read().decode('utf-8')

soup = BeautifulSoup(html, 'html.parser')

***

### 2. Tratamento preliminar da informação coletada:

In [4]:
# Separando as informações para tirar os espaços em excesso:


split = html.split()
join = " ".join(split)
replace = " ".join(html.split()).replace('> <', '><')
soup = BeautifulSoup(html, 'html.parser')

***

### 3. Variáveis para armazenar a informação coletada:

In [5]:
# Cada card (informações de 1 dos imóveis do site) será armazenado em um dicionário:

card = {}

# Será criada uma lista com todos os cards do site, isto é, cada item da lista será um card:
# Lembrando que cada card é representado pelo dicionário acima.

cards = []

***

### 4. Coleta das informações de 1 card (1 anúncio):

#### 4.1 Coleta das informações referentes ao preço do imóvel:

In [6]:
# Preço:

anuncio_preco = soup.find('li', {'class':'offer-item-price'}).get_text()
anuncio_preco.strip().replace(' ','')

'130000€'

In [7]:
# Tratamento da informação coletada:

preco=re.findall(r'[0-9]', anuncio_preco) 
preco = int(''.join(preco))
preco

130000

In [8]:
# Área:

anuncio_area = soup.find('li', {'class':'hidden-xs offer-item-area'}).get_text()
anuncio_area

'165 m²'

#### 4.1 Funções de formatação:

In [9]:
# Definição de função par tratamento de informações - texto >> numeral:

def texto_numero(texto):
    numero = re.findall(r'[0-9]', texto)
    numero = int(''.join(numero))
    return numero

In [10]:
# Conversão da informação em formato texto para o formato numérico conveniente:

area = texto_numero(anuncio_area)
area

165

In [11]:
# Preço por metro quadrado:

anuncio_m2 = soup.find('li', {'class':'hidden-xs offer-item-price-per-m'}).get_text()
anuncio_m2

'788 €/m²'

In [12]:
# Conversão da informação em formato texto para o formato numérico conveniente:

preco_m2 = texto_numero(anuncio_m2)
preco_m2

788

In [13]:
# Quantidade de quartos:

anuncio_quartos = soup.find('li', {'class':'offer-item-rooms hidden-xs'}).get_text()
anuncio_quartos

'T4'

In [14]:
# Conversão da informação em formato texto para o formato numérico conveniente:

quartos = texto_numero(anuncio_quartos)
quartos

4

In [15]:
anuncio_comp = soup.find('ul', {'class': 'parameters-view hidden-xs'}).get_text()
anuncio_comp.replace('\n', ' ').split()
anuncio_2 = anuncio_comp.replace('\n', ' ')
anuncio_2

' Casas de Banho: 4 ou mais Usado '

In [16]:
banheiros = texto_numero(anuncio_2)
banheiros

4

In [17]:
# Status do apartamento:

status = anuncio_2.replace('\n', ' ').split()[-1]
status

'Usado'

***

### 5. Criação do dicionário do 1º anúncio:

In [18]:
card['area'] = area

In [19]:
card['quartos'] = quartos

In [20]:
card['banheiros'] = banheiros 

In [21]:
card['preco'] = preco

In [22]:
card['preco_m2'] = preco_m2

In [23]:
card['status'] = status

In [24]:
card

{'area': 165,
 'quartos': 4,
 'banheiros': 4,
 'preco': 130000,
 'preco_m2': 788,
 'status': 'Usado'}

***

### 6. Lista de todos os anúncios da primeira página do site:

In [32]:
anuncios = soup.find('div', {'class' : 'col-md-content section-listing__row-content'}).find_all('article')

In [36]:
# Quantidade de anúncios por página:

len(anuncios)

24

In [34]:
# SOMENTE PARA VISUALIZAÇÃO:

# Pular 4 linhas entre um anúncio e outro:

for anuncio in anuncios:
    
    # Pulando 4 linhas entre um anúncio e outro:
    print(str(anuncio) + '\n\n\n\n')

<article class="offer-item ad_id170Ow" data-item-id="170Ow" data-tracking-id="16447763" data-url="https://www.imovirtual.com/pt/anuncio/apartamento-t4-5-assoalhadas-4-casas-de-banho-165m2-garagem-ID170Ow.html#49c4aa5237" id="offer-item-ad_id170Ow">
<figure class="offer-item-image" data-featured-name="highlight_7" data-quick-gallery='[{"photo":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6IjFyNzFla2M1OWdhZDMtRUNPU1lTVEVNIiwidyI6W3siZm4iOiI0amFmcGw5aDNmZ3QtQVBUIiwicyI6IjE0IiwicCI6IjEwLC0xMCIsImEiOiIwIn1dfQ.xb0XjJbSJZe32FSjmr_u6xVGyiVaDMxCpibX2HKCFhM\/image;s=655x491;q=80","thumb":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6IjFyNzFla2M1OWdhZDMtRUNPU1lTVEVNIiwidyI6W3siZm4iOiI0amFmcGw5aDNmZ3QtQVBUIiwicyI6IjE0IiwicCI6IjEwLC0xMCIsImEiOiIwIn1dfQ.xb0XjJbSJZe32FSjmr_u6xVGyiVaDMxCpibX2HKCFhM\/image;s=184x138;q=80"},{"photo":"https:\/\/ireland.apollo.olxcdn.com\/v1\/files\/eyJmbiI6IjRpYzNzMHNlMzRiaS1FQ09TWVNURU0iLCJ3IjpbeyJmbiI6IjRqYWZwbDloM2ZndC1BUFQiLCJzIjoiMTQiLCJwIjoiMT